https://mp.weixin.qq.com/s/2i7HxYAXvFJpWG3HKLr-OQ


重要概念

Summary：从TensorFlow获取数据

Tags：给数据一个name

Event Files & LogDirs：TensorBoard 从硬盘LogDirs路径的（Event Files）文件中加载数据

Runs：比较模型的不同执行情况


可视化面板介绍

Scalar Dashboard：记录标量变化，比如损失和学习率

Histogram Dashboard：显示张量的统计分布如何随时间变化

Distribution Dashboard：高级统计信息，可视化直方图数据

Image Dashboard：图像仪表板可以显示通过tf.summary.image保存的png

Audio Dashboard：音频仪表板可以嵌入可播放的音频小部件，以存储通过tf.summary.audio保存的音频，类似Image Dashboard

Graph Explorer：Graph Explorer可以可视化TensorBoard图，从而可以检查TensorFlow模型。

Embedding Projector：嵌入式投影仪可以可视化高维数据

Text Dashboard：文本仪表板，显示通过tf.summary.text保存的文本片段。支持超链接、列表和表等标记功能。

In [1]:
import tensorflow as tf

C:\Users\jianghaitao1\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\jianghaitao1\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\jianghaitao1\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\jian

In [2]:
tf.reset_default_graph()

In [3]:
a = tf.constant(2)
b = tf.constant(3)
c = tf.add(a,b)
with tf.Session() as sess:
    writer = tf.summary.FileWriter('../../dataset/logs/graphs',sess.graph)
    print(sess.run(c))

5


In [4]:
from tensorflow.examples.tutorials.mnist import input_data

In [5]:
# 载入数据
mnist = input_data.read_data_sets('../../dataset/mnist_data_1/',one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../../dataset/mnist_data_1/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../../dataset/mnist_data_1/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ../../dataset/mnist_data_1/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ../../dataset/mnist_data_1/t10k-labels-idx1-ubyte.gz
Instructions for updati

In [6]:
# 每个批次的大小，训练模型时，一次放入一批次
batch_size = 100
# 计算一共有多少个批次
n_batch = mnist.train.num_examples // batch_size

In [7]:
# 参数概要,定义一个函数，作用是计算各种参数值
def variable_summary(var):
    with tf.name_scope('summaries'):
        # 计算平均值
        mean = tf.reduce_mean(var)
        # summary.scalar用来显示标量信息
        tf.summary.scalar('mean',mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var-mean)))
        # 标准差
        tf.summary.scalar('stddev',stddev)
        # 最大值
        tf.summary.scalar('max',tf.reduce_max(var))
        # 最小值
        tf.summary.scalar('min',tf.reduce_min(var))
        # 直方图
        tf.summary.histogram('histogram',var)

In [8]:
# 命名空间
with tf.name_scope('input'):
    x = tf.placeholder(tf.float32,[None,784],name='x-input')
    y = tf.placeholder(tf.float32,[None,10],name='y-input')
# 创建一个简单的神经网络，输入层784个神经元，输出层10个神经元，不设隐藏层
with tf.name_scope('layer'):
    with tf.name_scope('wights'):
        # 权值，设一个变量，置0
        W = tf.Variable(tf.zeros([784,10]),name='W')
        # 把权值W当作参数，计算各种指标
        variable_summary(W)
    with tf.name_scope('biases'):
        # 偏置值
        b = tf.Variable(tf.zeros([10]),name='b')
        # 把偏置值b当作参数，计算各种指标
        variable_summary(b)
    with tf.name_scope('wx_plus_b'):
        wx_plus_b = tf.matmul(x,W) + b
    with tf.name_scope('softmax'):
        prediction = tf.nn.softmax(tf.matmul(x,W) + b)

In [9]:
# 使用交叉熵代价函数
with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction))
    tf.summary.scalar('loss',loss)
# 使用梯度下降法
with tf.name_scope('train'):
    train_step = tf.train.GradientDescentOptimizer(0.2).minimize(loss)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [10]:
# 初始化变量
init = tf.global_variables_initializer()

In [11]:
with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
        correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))
    with tf.name_scope('accuracy'):
        accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
        tf.summary.scalar('accuracy',accuracy)
# 合并所有的summary,并将其加入到sess.run的语句里
merged = tf.summary.merge_all()
with tf.Session() as sess:
    sess.run(init)
    writer = tf.summary.FileWriter('../../dataset/logs/graphs2',sess.graph)
    for epoch in range(n_batch):
        batch_xs,batch_ys = mnist.train.next_batch(batch_size)
        summary,_ = sess.run([merged,train_step],feed_dict={x:batch_xs,y:batch_ys})
        
        writer.add_summary(summary,epoch)
        acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels})
        print('Iter'+str(epoch)+',Testing Accuracy'+str(acc))

Iter0,Testing Accuracy0.1091
Iter1,Testing Accuracy0.1843
Iter2,Testing Accuracy0.2004
Iter3,Testing Accuracy0.2241
Iter4,Testing Accuracy0.3201
Iter5,Testing Accuracy0.4783
Iter6,Testing Accuracy0.3884
Iter7,Testing Accuracy0.3202
Iter8,Testing Accuracy0.3012
Iter9,Testing Accuracy0.3272
Iter10,Testing Accuracy0.4227
Iter11,Testing Accuracy0.4564
Iter12,Testing Accuracy0.4434
Iter13,Testing Accuracy0.5056
Iter14,Testing Accuracy0.5356
Iter15,Testing Accuracy0.5644
Iter16,Testing Accuracy0.5691
Iter17,Testing Accuracy0.5856
Iter18,Testing Accuracy0.5658
Iter19,Testing Accuracy0.5591
Iter20,Testing Accuracy0.555
Iter21,Testing Accuracy0.5641
Iter22,Testing Accuracy0.5347
Iter23,Testing Accuracy0.522
Iter24,Testing Accuracy0.5369
Iter25,Testing Accuracy0.5304
Iter26,Testing Accuracy0.5932
Iter27,Testing Accuracy0.5932
Iter28,Testing Accuracy0.6328
Iter29,Testing Accuracy0.6145
Iter30,Testing Accuracy0.6075
Iter31,Testing Accuracy0.5932
Iter32,Testing Accuracy0.5914
Iter33,Testing Accurac

Iter271,Testing Accuracy0.8102
Iter272,Testing Accuracy0.8074
Iter273,Testing Accuracy0.8094
Iter274,Testing Accuracy0.8095
Iter275,Testing Accuracy0.8113
Iter276,Testing Accuracy0.8113
Iter277,Testing Accuracy0.8106
Iter278,Testing Accuracy0.8106
Iter279,Testing Accuracy0.8105
Iter280,Testing Accuracy0.8115
Iter281,Testing Accuracy0.8121
Iter282,Testing Accuracy0.8116
Iter283,Testing Accuracy0.8118
Iter284,Testing Accuracy0.8119
Iter285,Testing Accuracy0.8119
Iter286,Testing Accuracy0.8124
Iter287,Testing Accuracy0.8125
Iter288,Testing Accuracy0.813
Iter289,Testing Accuracy0.8123
Iter290,Testing Accuracy0.8133
Iter291,Testing Accuracy0.8135
Iter292,Testing Accuracy0.8139
Iter293,Testing Accuracy0.8135
Iter294,Testing Accuracy0.814
Iter295,Testing Accuracy0.8146
Iter296,Testing Accuracy0.8146
Iter297,Testing Accuracy0.8146
Iter298,Testing Accuracy0.814
Iter299,Testing Accuracy0.8145
Iter300,Testing Accuracy0.814
Iter301,Testing Accuracy0.8131
Iter302,Testing Accuracy0.8131
Iter303,Test

Iter543,Testing Accuracy0.8245
Iter544,Testing Accuracy0.8246
Iter545,Testing Accuracy0.8246
Iter546,Testing Accuracy0.8247
Iter547,Testing Accuracy0.8248
Iter548,Testing Accuracy0.8243
Iter549,Testing Accuracy0.8242
